In [1]:
# install required packages
# Install the necessary dependencies
#First, let's install the necessary dependencies. In Google Colab, you can install Python packages using the `!pip install` command. 


!pip install python-dotenv PyPDF2 langchain openai tiktoken faiss-cpu

!pip install python-dotenv PyPDF2 langchain openai tiktoken faiss-cpu

In [2]:
# Import necessary libraries
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    pdf_reader = PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() if page.extract_text() else ""
    return text

# Function to split text into chunks
def split_text_into_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len
    )
    return text_splitter.split_text(text)

# Function to create a knowledge base from text chunks
def create_knowledge_base(chunks):
    embeddings = OpenAIEmbeddings()
    return FAISS.from_texts(chunks, embeddings)

# Function to get a user question (replacement for Streamlit input)
def get_user_question():
    user_question = input("Ask a question about your PDF: ")
    return user_question

# Function to get a response based on the user's question and the knowledge base
def get_response(knowledge_base, user_question):
    docs = knowledge_base.similarity_search(user_question)
    llm = OpenAI()
    chain = load_qa_chain(llm, chain_type="stuff")
    with get_openai_callback() as cb:
        response = chain.run(input_documents=docs, question=user_question)
        print(cb)
    return response

# Main function to orchestrate the PDF querying process
def main():
    # Assuming environment variables are set, if needed, before this script runs

    # File upload simulated by asking for file path in Colab environment
    pdf_path = input("Enter the path to your PDF file: ")
    if pdf_path:
        text = extract_text_from_pdf(pdf_path)
        if text:  # Check if text is not empty
            chunks = split_text_into_chunks(text)
            knowledge_base = create_knowledge_base(chunks)
            
            user_question = get_user_question()
            if user_question:  # Check if a question was entered
                response = get_response(knowledge_base, user_question)
                print("Response:", response)

if __name__ == '__main__':
    main()


/Users/undefined/dev/proj/ai/pdfreader/pdfreader/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/undefined/dev/proj/ai/pdfreader/pdfreader/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)